Discriminator와 Generator가 모두 어떤 추가적인 정보 y에 의해 conditioning 된다면 GAN은 조건부 모델로 확장 가능하다는 개념

$min\,max\,V(D,G) = E_{x~p_{data(x)}}[logD(x|y)]+E_{z~p_{z(z)}}[log(1-D(G(z|y)))]$

**Generator**\
차원(size)이 100인 prior noise z\
z와 y가 각각 size 200, 1000인 hidden layer(ReLu)로 mapping\
z와 y는 size 1200인 hidden layer로 합쳐짐(ReLu)\
마지막으로 784차원 MNIST 샘플을 생성하기 위한 출력으로 최종 sigmoid 단위 레이어가 존재 (784차원으로 변환됨, MNIST 이미지는 28^2)

**Discriminator**\
x를 240 unit와 5 piece로 구성된 maxout layer에 mapping\
y를 50 unit와 5 piece으로 구성된 maxout layer에 mapping\
240 unit, 4 piece의 maxout layer로 합쳐진 후 Sigmoid

모델\
크기가 100이고, 초기 학습률이 0.1\
모멘텀은 0.5 ~ 0.7 사용\
Dropout은 G와 D에 0.5로 적용\
검증 세트에 대한 로그 가능성의 최선 추정치를 중지점으로 사용함

가장 잘 작동하는 모델의 G\
크기 100의 가우스 노이즈를 prior noise로 수신?하여 500차원 ReLu layer에 mapping\
4096차원 이미지 특징 벡터를 2000차원 ReLu hidden layer에 mapping\
이 두 layer는 생성된 단어 벡터를 출력하는 200차원 선형 레이어의 공동 표현에 mapping


가장 잘 작동하는 모델의 D\
각각 단어 벡터 및 이미지 특징을 위한 500 및 1200차원 ReLu 은닉층과 1000단위 및 최종적으로 하나의 단일 시그모이드 단위에 공급되는 결합 층으로 3개 조각이 있는 최대 출력 층으로 구성됨

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio

In [13]:
batch_size = 64 # 배치 사이즈, 64
num_channels = 1 # 채널 수, 흑백 영상이므로 1?
num_classes = 10 # 클래스 수, 10
image_size = 28 # 이미지 차원, size, 28
latent_dim = 128

In [14]:
# mnist dataset 로드
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# x_train + x_test = all_digits
all_digits = np.concatenate([x_train, x_test])

# y_train + y_test = all_labels
all_labels = np.concatenate([y_train, y_test])

# 255로 나누어서 [0, 1] 값을 갖게 만듦.
all_digits = all_digits.astype("float32") / 255.0

# width = 28, height = 28, channel = 1으로 reshape함
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))

# to_categorical 사용하여 크기가 10으로 원-핫 인코딩
all_labels = keras.utils.to_categorical(all_labels, 10)

# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

Shape of training images: (70000, 28, 28, 1)
Shape of training labels: (70000, 10)


일반(무조건) GAN에서는 정규 분포에서 노이즈(일부 고정 차원)를 샘플링하는 것으로 시작합니다. 우리의 경우 클래스 레이블도 고려해야 합니다. 판별기(생성된 이미지 입력)뿐만 아니라 생성기(노이즈 입력)의 입력 채널에 클래스 수를 추가해야 합니다.

In [15]:
# z : latent_dim, 128
# y : num_classes, 클래스 수(10)
# x : num_channels, 채널 수(1)
generator_in_channels = latent_dim + num_classes # G(z|y)
discriminator_in_channels = num_channels + num_classes # D(x|y)
print(generator_in_channels, discriminator_in_channels) 

138 11


In [16]:
# Discriminator
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)), # 28 * 28
        # 컨볼루션 곱. filters=64, kernel_size=(3, 3), strides=(2, 2), padding="same"
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        # ReLU 활성화 함수
        layers.LeakyReLU(alpha=0.2),
        # 컨볼루션 곱. filters=128
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        # 풀링
        layers.GlobalMaxPooling2D(),
        # Dense : 뉴런의 입력과 출력을 연결해주는 역할
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense(7 * 7 * generator_in_channels), # 7 * 7
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # data 언패킹
        real_images, one_hot_labels = data

        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # generated_images : G가 random_vector_labels를 이용하여 생성한 이미지
        generated_images = self.generator(random_vector_labels)
        
        
        # generated_images와 원-핫 인코딩한 labels를 concat => fake_image_and_labels
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        # real_images와 원-핫 인코딩한 labels를 concat => real_image_and_labels
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )
        
        # labels 초기화? batch_size만큼 1과 0을 가진????
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # D 훈련
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            # D의 loss_fu이 d_loss? labels와 predictions
            d_loss = self.loss_fn(labels, predictions)
        # tape.gradient 사용하여 자동 미분하여 grads 구함
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        # 옵티마이저 거치고 gradients 적용?
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # batch size * latent_dim 만큼의 random_latent_vectors
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # 
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # G 학습 (D 고정?)
        with tf.GradientTape() as tape:
            # random_vector_labels 이용해서 fake_images 생성
            fake_images = self.generator(random_vector_labels)
            # fake_image_and_labels = fake_images + image_one_hot_labels
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            # 그 fake_image_and_lables 이용해서 D의 prediction 생성
            predictions = self.discriminator(fake_image_and_labels)
            # g_loss는 missleading_labels????와 predictions를 이용한 것
            g_loss = self.loss_fn(misleading_labels, predictions)
            
        # tape.gradient 사용하여 자동 미분하여 grads 구함
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        # 옵티마이저 거치고 gradients 적용?
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # g_loss 사용해서 G의 상태 업데이트?
        self.gen_loss_tracker.update_state(g_loss)
        # d_loss 사용해서 D의 상태 업데이트?
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [18]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    # BinaryCrossentropy: 실제 레이블과 예측 레이블 간의 교차 엔트로피 손실을 계산(loss_fn)
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=20)

Epoch 1/20
1094/1094 [==============================] - 945s 861ms/step - g_loss: 1.5388 - d_loss: 0.4112
Epoch 2/20
1094/1094 [==============================] - 677s 619ms/step - g_loss: 1.5309 - d_loss: 0.4309
Epoch 3/20
1094/1094 [==============================] - 734s 671ms/step - g_loss: 1.9389 - d_loss: 0.3263
Epoch 4/20
1094/1094 [==============================] - 754s 689ms/step - g_loss: 2.6134 - d_loss: 0.1778
Epoch 5/20
1094/1094 [==============================] - 803s 734ms/step - g_loss: 2.9037 - d_loss: 0.1897
Epoch 6/20
1094/1094 [==============================] - 836s 764ms/step - g_loss: 1.2051 - d_loss: 0.5565
Epoch 7/20
1094/1094 [==============================] - 828s 757ms/step - g_loss: 1.0425 - d_loss: 0.6276
Epoch 8/20
1094/1094 [==============================] - 21980s 20s/step - g_loss: 0.9151 - d_loss: 0.6521
Epoch 9/20
1094/1094 [==============================] - 1240s 1s/step - g_loss: 0.8615 - d_loss: 0.6633
Epoch 10/20
 403/1094 [==========>..............

KeyboardInterrupt: 

여기에서는 먼저 정규 분포에서 노이즈를 샘플링한 다음 이를 num_interpolation여러 번 반복하고 그에 따라 결과를 재구성합니다. num_interpolation 그런 다음 레이블 들여쓰기가 일정 비율로 존재하도록 균일하게 배포합니다 .

In [ ]:
# CGAN 형식으로 학습된 G인 trained_gen
trained_gen = cond_gan.generator

# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_interpolation = 9  # @param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = tf.cast(first_label, tf.float32)
    second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 1  # @param {type:"slider", min:0, max:9, step:1}
end_class = 5  # @param {type:"slider", min:0, max:9, step:1}

fake_images = interpolate_class(start_class, end_class)

In [ ]:
fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
converted_images = tf.image.resize(converted_images, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation.gif", converted_images, fps=1)
embed.embed_file("animation.gif")

FID 구하기

FID 점수는 사전 훈련된 Inception v3 모델을 먼저 로드하여 계산\
실제 이미지와 생성된 이미지에 대한 특징 벡터 컬렉션이 두 개 생성됨

$d^2 = ||mu_1 – mu_2||^2 + Tr(C_1 + C_2 – 2*sqrt(C_1*C_2))$

d^2는 점수\
mu_1, mu_2는 실제 이미지와 생성된 이미지의 특징별 평균\
C_1, C_2 는 실제 및 생성된 특징 벡터에 대한 공분산 행렬\
||mu_1 – mu_2||^2 는 두 평균 벡터 간의 차 제곱합\
Tr은 추적 선형 대수 연산 , 예를 들어 정사각형 행렬의 주대각선을 따라 요소의 합\
sqrt는 두 공분산 행렬 간의 곱으로 주어진 정사각 행렬의 제곱근

act1과 act2에 실제 이미지와 생성된 이미지의 특징 벡터 각각 넣어주면 됨

In [ ]:
# calculate frechet inception distance
def calculate_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

먼저, inception v3를 사용해서 이미지의 특징 벡터를 추출해야 함

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

inception_url = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
feature_model = tf.keras.Sequential([
    hub.KerasLayer(inception_url, output_shape=(2048,), trainable = False)
])
feature_model.build([None, 299, 299, 3])
feature_model.summary()

WandB

WandB(Weights & Biases)란 더 나은 모델을 빨리 만들 수 있도록 도와주는 머신러닝 Experiment tracking tool

In [ ]:
import wandb
import tensorflow as tf

wandb.init(project="my-test-project", entity="ujin2129")

In [ ]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

# ... Define a model

In [ ]:
with tf.Session() as sess:
  # ...
  wandb.tensorflow.log(tf.summary.merge_all())